In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import datetime
from pyspark.sql import Window
from pyspark.sql.types import *
import sys
import os
app_name = 'selection bu batch100'
# spark = SparkSession.builder.appName(app_name).enableHiveSupport().getOrCreate()
os.environ['PYSPARK_PYTHON'] = "/usr/bin/python3"
os.environ['PYSPARK_DRIVER_PYTHON'] = "/usr/bin/python3"

spark = (SparkSession\
         .builder\
         .appName("test")\
         .enableHiveSupport()\
         .config("spark.executor.instances", "150") \
         .config("spark.executor.memory", "32g") \
         .config("spark.executor.cores", "2") \
         .config("spark.driver.memory", "32g") \
         .config("spark.sql.shuffle.partitions", "8000") \
         .config("spark.default.parallelism", "8000") \
         .config("spark.driver.maxResultSize", "32g") \
         .config("spark.pyspark.python", "/usr/bin/python3")\
         .config("spark.sql.autoBroadcastJoinThreshold","-1")
         .config("spark.yarn.appMasterEnv.yarn.nodemanager.container-executor.class","DockerLinuxContainer")\
         .config("spark.executorEnv.yarn.nodemanager.container-executor.class","DockerLinuxContainer")\
         .config("spark.yarn.appMasterEnv.yarn.nodemanager.docker-container-executor.image-name","bdp-docker.jd.com:5000/wise_mart_rmb_py36:latest")\
         .config("spark.executorEnv.yarn.nodemanager.docker-container-executor.image-name","bdp-docker.jd.com:5000/wise_mart_rmb_py36:latest")\
         .getOrCreate())


spark.conf.set("hive.exec.dynamic.partition", "true")
spark.conf.set("hive.exec.dynamic.partition.mode", "nonstrict")

import datetime
import sys
import pandas as pd
import os
import numpy as np
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
import pyspark.sql.types as sql_type
import spa_utils
name = locals()

## 指定更新的日期

In [3]:
# 每天跑前一天的
# 每个时间都可以改，update_start和update_end是需要插入的日期，update_origin必须为update_start的前三个月的日期
import datetime
import time
yesterday = (datetime.datetime.now()-datetime.timedelta(days=1)).strftime('%Y-%m-%d')
start_day = (datetime.datetime.strptime(spark.sql('''select max(dt) from app.app_pa_price_baseprice_self_deal_price_60_5 ''').collect()[0][0],'%Y-%m-%d')+datetime.timedelta(days=1)).strftime('%Y-%m-%d')
origin_day = (datetime.datetime.strptime(start_day,'%Y-%m-%d')-91*datetime.timedelta(days=1)).strftime('%Y-%m-%d')
params = {
    'update_origin':origin_day,
    'update_start':start_day,
    'update_end':yesterday
}
update_origin = datetime.datetime.strptime(params['update_origin'], '%Y-%m-%d')
update_start = params['update_start']
update_end = datetime.datetime.strptime(params['update_end'], '%Y-%m-%d')

percent = 0.97

## 60天-红价填充空值再算基线价--去掉双十一618-改成固定的60*0.3

In [4]:
def get_median(data):
    data.sort()
    half = len(data) // 2
    return (data[half] + data[~half]) / 2

def base_price(price_list):
    price_list_sorted = sorted(price_list,reverse = True)
    batch_mean_price = []
    batch_num_count = []
    base_price =999999999999
    while price_list_sorted:
        batch = []
        for i in range(len(price_list_sorted)):
            if round(price_list_sorted[i]-0.97*price_list_sorted[0],4)>=0:
                batch.append(price_list_sorted[i])
                max_i = i
#                 print('batch:%s'%batch)
#                 print('max_i%s'%max_i)
        batch_mean_price.append(sum(batch) / len(batch))
        batch_num_count.append(max_i+1)
        price_list_sorted = price_list_sorted[max_i+1:]
#         print('batch_mean_price%s'%batch_mean_price)
#         print('batch_num_count%s'%batch_num_count)  
#         print('price_list_sorted%s'%price_list_sorted)  
    print(batch_mean_price,batch_num_count)
    sum_batch_num_count =  sum(batch_num_count)
#    print('sum_batch_num_count%s'%sum_batch_num_count) 
    batch_mean_copy = batch_mean_price.copy()
    medium = get_median(batch_mean_copy)
    ks = [[x,y] for x,y in zip(batch_mean_price, batch_num_count) if x>=medium*0.1]
#     batch_mean_price = list(filter(lambda x: x>medium*0.1,batch_mean_price))      
#     batch_num_count = list(filter(lambda x: x>medium*0.1,batch_num_count))
    batch_mean_price = [x[0] for x in ks]
    batch_num_count = [x[1] for x in ks]
#     print('batch_mean_price:%s'%batch_mean_price)
#     print('batch_num_count:%s'%batch_num_count)
    for i in range(len(batch_num_count)):
        if batch_num_count[i] > 0.3 * 60 and batch_mean_price[i] < base_price:
            base_price = batch_mean_price[i]
            print('base_price_0%s'%base_price)      
    if base_price == 999999999999:
        for i in range(len(batch_num_count)):
            if batch_num_count[i] == max(batch_num_count) and batch_mean_price[i] < base_price:
                base_price = batch_mean_price[i]
#         print('a_index%s'%a.index(max(a)))
#         print('base_price_1%s'%base_price)      
    return(base_price)

### 红价填充成交价没有的天数

In [5]:
# 读取成交价与红价，已经处理为(sku, date)粒度，包含全部日期
deal = spark.sql('''
select item_sku_id as item_sku_id, after_prefr_amount,dq_pay_amount,sale_qtty, dt from %s where dt>='%s' and dt<='%s'
''' % ('app.app_pa_transactions_d_self', params['update_origin'],params['update_end']))
deal = deal.groupby('item_sku_id','dt').agg(((F.sum('after_prefr_amount')-F.sum('dq_pay_amount'))/F.sum('sale_qtty')).alias('price'),
                                        F.sum('sale_qtty').alias('sale_qtty'))
deal = deal.withColumn('price',F.when(F.col('price')<0,F.round(F.col('price'),0)).otherwise(F.col('price')))\
       .filter(F.col('price')>=0)

redprice = spark.sql('''
select sku_id as item_sku_id, max_price as scrapedprice, dt from %s where dt>='%s' and dt<='%s'
''' % ('dev.self_sku_redprice_group', params['update_origin'],params['update_end']))

# 读取sku上下柜情况
df_status = spark.sql('''
select sku_id as item_sku_id, sku_status_cd, dt from %s where sku_type=1 and dt>='%s' and dt<='%s'
''' % ('dev.self_sku_det_da', params['update_origin'],params['update_end'])).distinct()
# 只保留当天上柜的sku
df_status = df_status.filter(df_status.sku_status_cd == 3001)

df = df_status.join(redprice, ['item_sku_id', 'dt'], 'inner').join(deal,['item_sku_id','dt'],'left')
df = df.withColumn('price',F.when(F.col('price').isNull(),F.col('scrapedprice')).otherwise(F.col('price')))\
       .select('item_sku_id','dt','price','sale_qtty')
df.cache()
df.count()

269313682

## 更新新一天的基线价

In [6]:
spark.sql("set hive.exec.dynamic.partition=true")
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
spark.sql('''set hive.exec.max.dynamic.partitions=200551''')
spark.sql('''set hive.exec.max.dynamic.partitions.pernode=200551''')
table_params = {'author':'zoushuhan'}

print(time.time())
days = 60
base_0 = df.filter(F.col('dt')>=update_start)\
           .select('item_sku_id','dt')\
           .withColumn('dt_past',F.date_sub(F.col('dt'), 0))

for ii in range(1,days+1):
    name['base_%s'%ii] = df.filter(F.col('dt')>=update_start).select('item_sku_id','dt').withColumn('dt_past',F.date_sub(F.col('dt'), ii))
    base_0 = base_0.union(eval('base_%s'%ii))
# 去掉双十一和618再去计算基线价
base_0 = base_0.filter((F.substring(F.col('dt_past'),6,5)<='05-31')|((F.substring(F.col('dt_past'),6,5)>='06-21')&(F.substring(F.col('dt_past'),6,5)<='10-31'))|
                       (F.substring(F.col('dt_past'),6,5)>='11-14'))
df1 = base_0.join(df.withColumnRenamed('dt','dt_past').select('item_sku_id','dt_past','price'),['item_sku_id','dt_past'],'inner') 
df1 = df1.withColumn('price_1',F.col('price').cast('Float'))

grouped_df = df1.groupby("item_sku_id","dt").agg(F.collect_list("price_1").alias("list_price"))


# grouped_df = grouped_df.withColumn('dt1',F.col('dt')).drop('dt').withColumnRenamed('dt1','dt')
# spa_utils.save_hive_result(grouped_df,'app.app_pa_price_baseprice_self_deal_price_temp',partitioning_columns=['dt'],write_mode='save',spark=spark,params=table_params)

1570777604.8945994


In [6]:
# grouped_df = spark.sql('''select * from app.app_pa_price_baseprice_self_deal_price_temp ''')
base_price_udf = F.udf(base_price,FloatType())
df2 = grouped_df.withColumn('baseprice', base_price_udf('list_price'))
df2 = df2.select('item_sku_id','baseprice','dt')

spa_utils.save_hive_result(df2,'app.app_pa_price_baseprice_self_deal_price_60_5',partitioning_columns=['dt'],write_mode='insert',spark=spark,params=table_params)
# spark.sql('''drop table if exists app.app_pa_price_baseprice_self_deal_price_temp ''')
print(time.time())

1570761236.7333064
